In [ ]:
import darklight
import tangos

Import the simulation you'd like to run DarkLight on.  The following assumes that the database `Halo600.db` is in your current working directory.

If analzying EDGE halos, see `tutorial-edge.ipynb` for notes on additional support in DarkLight for retrieving EDGE data on DiRAC or the Surrey servers. 

In [ ]:
tangos.core.init_db('Halo600.db')
sim = tangos.get_simulation('Halo600_fiducial')

This simulation contains lots of dark matter halos, but one giant 'main' halo.  The rest of the halos are much smaller. The simulations are run for the age of the universe, ~13.8 Gyr, and saves a bunch of 'snapshots' of the halo as it evolves over time. 

What we do here is get the last snapshot (timesteps[-1]), which corresponds to the present day (i.e. a redshift z = 0), and get the most massive halo (halos[0]), which will always be the first halo in the list of all halos, which is ordered by mass.

In [ ]:
halo = sim.timesteps[-1].halos[0]

Now that we have the halo, we can run DarkLight on it!

In [ ]:
t,z,vsmooth,sfh_insitu,mstar_insitu,mstar_total = darklight.DarkLight(halo)

The outputs are:

 - `t` = times [Gyr]
 - `z` = redshifts [unitless]
 - `vsmooth` = vmax over time, smoothed [km/s]
 - `sfh_insitu` = the star formation history of the galaxy that forms in this halo [Msun/yr]
 - `mstar_insitu` = integral of the (cumulative) in-situ star formation history [Msun]
 - `mstar_total` = total mass formed insitu + was accreted as little halos merged with this halo

All of these output arrays are of the same length.

I've created a function that takes the outputs from DarkLight and plots it against the values from the full hydrodynamic simulations. The stellar masses calculated by DarkLight usually match the values from the simulations to within a factor of two or so, but won't be an exact match.

In [ ]:
darklight.edge.plot_darklight_vs_edge_mstar(halo,t,z,vsmooth,sfh_insitu,mstar_total,mstar_insitu=mstar_insitu,figfn='halo'+shortname+'.pdf')

In the above example, I've run it with the default DarkLight settings, but you can play with these settings by supplying some keyword arguments. Here's some of the keyword arguments it takes that changes the model that goes into DarkLight, and their default values.  Note that vmax = the maximum velocity in the rotation curve.

 - `zre = 4`, the redshift reionization shuts star formation off
 - `vthres = 26.3` [km/s], minimum vmax the halo must have to start forming stars again after reionization
 - `occupation = 2.5e7` [msun] the occupation fraction (i.e. what halos have galaxies) to assume.  By default, assumes all halos with a mass above the given value (in solar masses) have galaxies.  However, you can also ask DarkLight to assume all halos have galaxies (`'all'`), or use the occupation function from the EDGE1 sims (`'edge1'`), the EDGE1 RT sims (`'edge1rt'`), or a fit to observed Milky Way dwarfs (`'nadler20'`).
 - `pre_method = 'fiducial'`, the SFR-vmax relation to use before reionization
 - `post_method = 'schechter'`, the SFR-vmax relation to use after reionization
 - `post_scatter_method='increasing'`, what scatter to assume for the SFR-vmax relation after reionization

There's also a few other parameters that you can modify:

 - `nscatter` = 0, the number of realizations of the halo's SFH to generate; by default, DarkLight generates one, trajectory, but setting this > 1 will generate that number of SFHs and M*(t)
 - `timesteps = sim` [Gyr], the time resolution to produce the SFH and thus M*(t), which by default will use the timesteps in the simulation.  You can instead also specify a timestep in units of Gyr.
 - `binning = 3bins`, which assigns single SFR before reionization based on its average vmax in that period (bin 1), zero after reionization when the halo's vmax < vthres (bin 2), and a single SFR if vmax > vthres based on its vmax at z=0 (bin 3).  You can also specify `'all'` to have it update the SFR based on the vmax at each timestep. 
 - `DMO = False`, if you're running on a hydro sim or a dark matter only (DMO) sim.  If running on a DMO sim, multiplies particle masses by sqrt(1-fbary).
 - `mergers = True`, whether or not to include the contribution to M* from in-situ star formation, mergers, or both.  `True` includes both, `False` only the in-situ stars, and `'only'` only the accreted stars.

Generally, you'll find that if you run with multiple realizations, the average of all those runs will better match the EDGE simulations than if you ran it only once.  Here's an example running DarkLight with multiple realizations:

In [ ]:
t,z,vsmooth,sfh_insitu,mstar_insitu,mstar_total = darklight.DarkLight(halo, nscatter=100)
darklight.edge.plot_darklight_vs_edge_mstar(halo,t,z,vsmooth,sfh_insitu,mstar_total,mstar_insitu=mstar_insitu,figfn='halo'+shortname+'-scatter.pdf')